Okay, here's a comprehensive solution that integrates your Jupyter Notebook code into a Flask application, makes it deployable on GitHub Pages, and allows users to select a date.  I'll break this down into several parts:

**1. Project Structure:**

We'll organize the project like this:

```
fire-weather-app/
├── .github/
│   └── workflows/
│       └── deploy.yml  (This is the workflow file)
├── app.py               (Flask application)
├── functions.py        (Functions from your notebook)
├── requirements.txt     (Dependencies)
├── static/
│   └── index.html       (Generated map - will be overwritten)
└── templates/
    └── index.html       (Flask template for user input)

```

**2. `app.py` (Flask Application):**

This is the core of the Flask web application.

```python
from flask import Flask, render_template, request
import functions  # Import your notebook functions
import pandas as pd
import os

app = Flask(__name__)

# Set a secret key for session management (important for security)
app.secret_key = os.urandom(24)  # Generate a random secret key


@app.route("/", methods=["GET", "POST"])
def index():
    map_html = None  # Initialize map_html
    selected_date = None

    if request.method == "POST":
        selected_date_str = request.form["date"]
        try:
            # Convert the selected date string to a datetime object
            selected_date = pd.to_datetime(selected_date_str)
        except ValueError:
            return render_template("index.html", error="Invalid date format.  Please use YYYY-MM-DD.")


        # Generate or update the map
        map_html = functions.generate_map_for_date(selected_date)

        if map_html:
             # Save the map to the static folder (for GitHub Pages)
            with open("static/index.html", "w", encoding="utf-8") as f:
                f.write(map_html)


    # Use current day if not set yet
    if not selected_date:
        selected_date = pd.to_datetime('today')


    return render_template("index.html", map_html=map_html, selected_date=selected_date.strftime("%Y-%m-%d"))


if __name__ == "__main__":
    app.run(debug=True)  # Use debug=True for development, False for production
```

**3. `functions.py` (Notebook Functions):**

This file will contain the core logic extracted from your Jupyter Notebook.  This promotes reusability and keeps `app.py` clean.


Here's a precise breakdown of what to copy into `functions.py`, along with explanations:

**Copying from Your Notebook to `functions.py`**

1.  **Part 0: Initial Setup:** Copy *everything* from Part 0. This includes all the `import` statements and the following functions:
    *   `get_california_geojson_from_cnra_api`
    *   `is_point_in_la_county`
    *   `create_la_county_base_map`

2.  **Part 1: Fire Weather Index Calculation Functions:** Copy *everything* from Part 1.  This includes:
    *   `get_weather_data`
    *   `calculate_fire_danger_nfdrs4`
    *   `calculate_kbdi` (this is defined *inside* `calculate_fire_danger_nfdrs4` in the original code; copy the entire outer function).
    *   `kbdi_to_df` (also inside `calculate_fire_danger_nfdrs4`)
    *   `calculate_ffdi` (also inside `calculate_fire_danger_nfdrs4`)
    *   `fine_fuel_moisture_code`
    *   `duff_moisture_code`
    *   `drought_code`
    *   `initial_spread_index`
    *   `buildup_index`
    *   `fire_weather_index`
    *   `fwi_from_dataframe`
    *    `calculate_kbdi` (The stand alone method, not part of another method, from the end of Part 1 of provided code)
    *    `calc_ffwi`

3.  **Part 2: Map Generation with Folium:**  Copy *everything* from Part 1, particularly the `create_fire_index_map` function.  

4.  **Part 3: Main Execution Block:** Copy *nothing* directly from the main execution block of Part 3 *into* `functions.py`.  Instead, we'll create a *new* function in `functions.py` called `generate_map_for_date`.  This new function encapsulates the map generation logic that was originally in the main execution block, but it's now parameterized by date.  This is crucial for the Flask integration.

**New Function: `generate_map_for_date`**

Add this function to `functions.py`. This is the key function that brings everything together and is called by the Flask app.

```python
def generate_map_for_date(selected_date):
    """Generates the Folium map HTML for a given date."""

    combined_map, la_county_layer, base_map_layer = create_la_county_base_map()
    if combined_map is None or la_county_layer is None:
        print("Failed to create base map. Exiting.")
        return None  # Or return an error message/HTML

    # Ensure selected_date is a string in the correct format
    date_str = selected_date.strftime("%Y-%m-%d")

    grid_data = {}
    grid_spacing = 0.1
    # Define the boundaries for Los Angeles County (approximate)
    min_lat, max_lat = 33.28, 34.86  # Latitude range
    min_lon, max_lon = -119.1, -117.3  # Longitude range
    extended_min_lon = min_lon - 6 * grid_spacing
    extended_max_lon = max_lon + 2 * grid_spacing

    la_county_grid = [
        (lat, lon)
        for lat in np.arange(min_lat, max_lat + grid_spacing, grid_spacing)
        for lon in np.arange(extended_min_lon, extended_max_lon + grid_spacing, grid_spacing)
    ]

    for lat, lon in la_county_grid:
        try:
            current_weather_data, daily_weather_data = get_weather_data(lat, lon, date_str, date_str)
            tmax = daily_weather_data["temperature_2m_max"]
            rhmin = daily_weather_data["relative_humidity_2m_min"]
            wind_speed_mph = current_weather_data["wind_speed"] * 0.621371
            precipitation_inches = daily_weather_data["precipitation_sum"] * 0.0393701

            nfdrs4_data = calculate_fire_danger_nfdrs4(
                current_weather_data["temperature"],
                tmax,
                current_weather_data["relative_humidity"],
                rhmin,
                precipitation_inches,
                wind_speed_mph,
            )
            fwi_df = pd.DataFrame(
                {
                    "temperature": [current_weather_data["temperature"]],
                    "relative_humidity": [current_weather_data["relative_humidity"]],
                    "wind_speed": [current_weather_data["wind_speed"]],
                    "precipitation": [current_weather_data["precipitation"]],
                    "latitude": [lat],
                }
            )
            fwi_result = fwi_from_dataframe(fwi_df, mon=selected_date.month)
            mffwi_ffwi, mffwi_val = calc_ffwi(
                tmax * 9 / 5 + 32, rhmin, wind_speed_mph, kbdi=nfdrs4_data["KBDI"]
            )
            grid_data[(lat, lon)] = {
                "FFDI": nfdrs4_data["Forest Fire Danger Index (FFDI)"],
                "FWI": fwi_result["FWI"].iloc[0],
                "mFFWI": mffwi_val if mffwi_val is not None else None,
            }
        except Exception as e:
            print(f"Error processing data for ({lat}, {lon}): {e}")
            grid_data[(lat, lon)] = {"FFDI": None, "FWI": None, "mFFWI": None}


    # --- Map creation (similar to your original Part 3) ---
    la_county_layer.add_to(combined_map)
    base_map_layer.add_to(combined_map)

    # Create feature groups for each index combination and add them to the map
    index_combinations = [
        'FWI', 'FFDI', 'mFFWI', 'FWI-FFDI', 'FWI-mFFWI', 'FFDI-mFFWI',
        'FWI-FFDI-mFFWI'
    ]
    base_layers = {}  # Dictionary to store base layers
    for index_combination in index_combinations:
        base_map, layer = create_fire_index_map(grid_data, index_combination, combined_map)
        feature_group = folium.FeatureGroup(name=index_combination, control=True, overlay=False)
        feature_group.add_to(combined_map)
        layer.add_to(feature_group) #add to feature group
        base_layers[index_combination] = feature_group  # Store FeatureGroup

    # --- Colormap handling (important for display) ---

    ffdi_colormap = folium.LinearColormap(
    colors=['green', 'yellow', 'orange', 'red'],
    vmin=min(data['FFDI'] for data in grid_data.values() if data['FFDI'] is not None),
    vmax=max(data['FFDI'] for data in grid_data.values() if data['FFDI'] is not None),
    caption='FFDI Value'
    )
    fwi_colormap = folium.LinearColormap(
    colors=['green', 'yellow', 'orange', 'red'],
    vmin=min(data['FWI'] for data in grid_data.values() if data['FWI'] is not None),
    vmax=max(data['FWI'] for data in grid_data.values() if data['FWI'] is not None),
    caption='FWI Value'
    )
    mffwi_colormap = folium.LinearColormap(
    colors=['green', 'yellow', 'orange', 'red'],
    vmin=min(data['mFFWI'] for data in grid_data.values() if data['mFFWI'] is not None),
    vmax=max(data['mFFWI'] for data in grid_data.values() if data['mFFWI'] is not None),
    caption='mFFWI Value'
    )
    # Get HTML representation of the colormaps
    ffdi_colormap_html = ffdi_colormap._repr_html_()
    fwi_colormap_html = fwi_colormap._repr_html_()
    mffwi_colormap_html = mffwi_colormap._repr_html_()

    # Format numbers in colormap HTML to two decimal places
    ffdi_colormap_html = re.sub(r"(\d+\.\d{2})\d+", r"\1", ffdi_colormap_html)
    fwi_colormap_html = re.sub(r"(\d+\.\d{2})\d+", r"\1", fwi_colormap_html)
    mffwi_colormap_html = re.sub(r"(\d+\.\d{2})\d+", r"\1", mffwi_colormap_html)

    # Remove the min and max value lines from colormap HTML - ADJUSTED for direct string replacement
    ffdi_colormap_html = ffdi_colormap_html.replace('<li style="text-align: center; list-style: none; display: block; line-height: 18px; height: 18px; width: 100.0%; margin-left: 0%; margin-bottom: -2px;"><span style="text-align: right; display: block; width: 40px; float: left; height: 100.0%; background: rgba(0,0,0,0);">{:.2f}</span><span style="display: block; width: 5.0%; float: left; height: 100.0%; background: rgba(0,0,0,0);"><svg class="colorbar" height="18" width="10.0" style="float: right;"><line stroke-width="1" x1="0" x2="0" y1="1" y2="16" stroke="#000000"></line></svg></span>', '').replace('<span style="display: block; width: 5.0%; float: left; height: 100.0%; background: rgba(0,0,0,0);"><svg class="colorbar" height="18" width="10.0" style="float: right;"><line stroke-width="1" x1="0" x2="0" y1="1" y2="16" stroke="#000000"></line></svg></span><span style="display: block; width: 40px; float: left; height: 100.0%; background: rgba(0,0,0,0);">{:.2f}</span></li>', '')
    fwi_colormap_html = fwi_colormap_html.replace('<li style="text-align: center; list-style: none; display: block; line-height: 18px; height: 18px; width: 100.0%; margin-left: 0%; margin-bottom: -2px;"><span style="text-align: right; display: block; width: 40px; float: left; height: 100.0%; background: rgba(0,0,0,0);">{:.2f}</span><span style="display: block; width: 5.0%; float: left; height: 100.0%; background: rgba(0,0,0,0);"><svg class="colorbar" height="18" width="10.0" style="float: right;"><line stroke-width="1" x1="0" x2="0" y1="1" y2="16" stroke="#000000"></line></svg></span>', '').replace('<span style="display: block; width: 5.0%; float: left; height: 100.0%; background: rgba(0,0,0,0);"><svg class="colorbar" height="18" width="10.0" style="float: right;"><line stroke-width="1" x1="0" x2="0" y1="1" y2="16" stroke="#000000"></line></svg></span><span style="display: block; width: 40px; float: left; height: 100.0%; background: rgba(0,0,0,0);">{:.2f}</span></li>', '')
    mffwi_colormap_html = mffwi_colormap_html.replace('<li style="text-align: center; list-style: none; display: block; line-height: 18px; height: 18px; width: 100.0%; margin-left: 0%; margin-bottom: -2px;"><span style="text-align: right; display: block; width: 40px; float: left; height: 100.0%; background: rgba(0,0,0,0);">{:.2f}</span><span style="display: block; width: 5.0%; float: left; height: 100.0%; background: rgba(0,0,0,0);"><svg class="colorbar" height="18" width="10.0" style="float: right;"><line stroke-width="1" x1="0" x2="0" y1="1" y2="16" stroke="#000000"></line></svg></span>', '').replace('<span style="display: block; width: 5.0%; float: left; height: 100.0%; background: rgba(0,0,0,0);"><svg class="colorbar" height="18" width="10.0" style="float: right;"><line stroke-width="1" x1="0" x2="0" y1="1" y2="16" stroke="#000000"></line></svg></span><span style="display: block; width: 40px; float: left; height: 100.0%; background: rgba(0,0,0,0);">{:.2f}</span></li>', '')

    colormap_div = folium.Element(
        f"<div id='colormap' style='position: absolute; bottom: 30px; left: 5px; display: inline-flex; gap: 16px; z-index: 1000; background-color: rgba(255, 255, 255, 0.7); width: 99%;'>"
        + ffdi_colormap_html
        + fwi_colormap_html
        + mffwi_colormap_html
        + "</div>"
    )
    combined_map.get_root().html.add_child(colormap_div)

    # Add LayerControl (MUST be after adding feature groups)
    control = folium.LayerControl(collapsed=False, exclusive_groups=["Fire Indices"])
    combined_map.add_child(control)

    # Return the HTML representation of the map
    return combined_map.get_root().render()

```

Key changes and explanations within `generate_map_for_date`:

*   **Date Parameter:**  Takes `selected_date` (a `datetime` object) as input.
*   **Date Formatting:**  Formats the date correctly for the Open-Meteo API (`YYYY-MM-DD`).
*   **Error Handling:** Includes a `try-except` block to handle potential errors during data fetching and processing.  If an error occurs for a specific grid point, it assigns `None` values to the fire indices for that point, preventing the entire map generation from failing.
*   **Map HTML Return:**  Instead of saving the map to a file, it returns the *HTML representation* of the map using `combined_map.get_root().render()`.  This is what Flask will use to display the map.
*   **Colormap Handling:** The colormap HTML is generated and added to the map, *but* the min/max value removal is done using direct string replacement, which is more robust. The colormap is added *before* the LayerControl.
*   **Layer Control:** The `folium.LayerControl` is added *after* all the feature groups have been added to the map.

**4. `templates/index.html` (Flask Template):**

This HTML file provides the user interface for date selection.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Fire Weather Map</title>
    <style>
        body { font-family: sans-serif; }
        .map-container { width: 100%; height: 500px; } /* Adjust as needed */
        .error-message { color: red; }
    </style>
</head>
<body>
    <h1>Fire Weather Map - Los Angeles County</h1>

    <form method="POST">
        <label for="date">Select Date:</label>
        <input type="date" id="date" name="date" value="{{ selected_date }}" required>
        <button type="submit">Generate Map</button>
    </form>

    {% if error %}
        <p class="error-message">{{ error }}</p>
    {% endif %}

     {% if map_html %}
    <div class="map-container" id="map">
    </div>

     <script>
            // Wait for the DOM to be fully loaded
            document.addEventListener('DOMContentLoaded', function() {
                // Find the map container element
                var mapContainer = document.getElementById('map');

                // Set the innerHTML of the map container with your map_html
                mapContainer.innerHTML = `{{ map_html|safe }}`;
            });
        </script>
    {% else %}
    <p>Please select a date, and then select the "Generate Map" button, to view the map.</p>
    {% endif %}
</body>
</html>
```

Key features of `templates/index.html`:

*   **Date Input:**  Uses an HTML `<input type="date">` element, which provides a nice date picker in most browsers.
*   **Form:**  Uses a `<form>` with `method="POST"` to send the selected date to the Flask server.
*   **Error Handling:** Displays an error message (`{% if error %}`) if the date is invalid.
*    **Conditional Map Display:**  The `{% if map_html %}` block only displays the map if `map_html` is not `None`.  This prevents errors if the map generation fails.  The `{{ map_html|safe }}` part is *crucial*.  It tells Jinja2 (Flask's templating engine) to render the `map_html` as raw HTML, *without* escaping the HTML tags.  Without `|safe`, the map wouldn't display.
*   **Map Container:** A `<div>` with `class="map-container"` is used to hold the map.  The CSS sets its size.  You can adjust the `width` and `height` as needed.
* **JavaScript Insertion of Map**: The JavaScript code waits until all DOM content is loaded, then adds the `map_html` to the DOM.

**5. `requirements.txt`:**

This file lists the Python packages your project needs.

```
Flask
requests
folium
requests-cache
pandas
retry-requests
numpy
shapely
openmeteo-requests
Jinja2
branca
```

**6. Deployment to GitHub Pages:**

1.  **Commit and Push:**  Commit all your files (`app.py`, `functions.py`, `requirements.txt`, `static/index.html`, `templates/index.html`) to your GitHub repository.  Make sure they are in the *root* of your repository (not inside a `docs` folder, as you would for a static site).

2.  **GitHub Pages Settings:**
    *   Go to your repository's settings on GitHub.
    *   Scroll down to the "GitHub Pages" section.
    *   For "Source", select "Deploy from a branch".
    *   For "Branch", select the branch you're working on (e.g., `main` or `master`).
    *   Set the folder to `/ (root)`.
    *   Click "Save".

3.  **Important: GitHub Actions (for dynamic content):** Because your site is *dynamic* (it uses Flask), GitHub Pages won't automatically run your `app.py`. You need to use GitHub Actions to build and deploy your site.  This involves creating a workflow file.

    Create a file named `.github/workflows/deploy.yml` (the folder structure is important) in your repository with the following content:

    ```yaml
    name: Deploy to GitHub Pages

    on:
      push:
        branches:
          - main  # Or your default branch name

    jobs:
      deploy:
        runs-on: ubuntu-latest
        steps:
          - name: Checkout code
            uses: actions/checkout@v3

          - name: Set up Python
            uses: actions/setup-python@v4
            with:
              python-version: '3.10'  # Or your desired Python version

          - name: Install dependencies
            run: |
              python -m pip install --upgrade pip
              pip install -r requirements.txt

          - name: Run Flask app (to generate static/index.html)
            run: python app.py

          - name: Deploy to GitHub Pages
            uses: peaceiris/actions-gh-pages@v3
            with:
              github_token: ${{ secrets.GITHUB_TOKEN }}
              publish_dir: ./static  # Deploy the static directory
              #cname: yourcustomdomain.com  # Optional: If you have a custom domain
    ```

    *   **`on: push:`:**  This triggers the workflow whenever you push to the `main` branch (change `main` if you use a different branch).
    *   **`runs-on: ubuntu-latest`:** Specifies the operating system for the runner.
    *   **`actions/checkout@v3`:** Checks out your code.
    *   **`actions/setup-python@v4`:** Sets up the Python environment.
    *   **`pip install -r requirements.txt`:** Installs your dependencies.
    *   **`python app.py`:**  This is *crucial*. It runs your Flask application *once*.  This is enough because your `app.py` is designed to generate the `static/index.html` file, which is what GitHub Pages will serve. We *don't* need a continuously running server.
    *   **`peaceiris/actions-gh-pages@v3`:** Deploys the `static` directory to GitHub Pages.
    *   **`publish_dir: ./static`:**  This tells the action to deploy the contents of the `static` folder.  This is where your generated map (`index.html`) is located.

4.  **Access Your Site:** After the GitHub Actions workflow completes (you can monitor its progress in the "Actions" tab of your repository), your site will be live at `https://<your-github-username>.github.io/<your-repository-name>/`.


The initial static/index.html file can be almost anything, as it will be overwritten by the Flask app. It's good practice to have a placeholder file, though, so that GitHub Pages has something to serve initially, before GitHub Actions workflow runs for the first time.

Here's a simple placeholder that can be used for the initial static/index.html:

```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Loading...</title>
</head>
<body>
    <h1>Loading Fire Weather Map...</h1>
    <p>Please wait while the map is being generated.</p>
</body>
</html>
```

Explanation:

    Placeholder Content: This provides a user-friendly message while the actual map is being generated.

    Overwritten: This file's content will be completely replaced by the output of your Flask app (specifically, the output of the generate_map_for_date function) each time the GitHub Actions workflow runs.

    GitHub Pages Initial State: When you first set up GitHub Pages, it looks for an index.html file in the specified directory (in this case, static). If it doesn't find one, there could be a 404 error. This placeholder ensures that GitHub Pages has something to display until your app takes over.

    Do Not Put Map Logic Here: This initial file should not contain any mapping code, colormap definitions, or JavaScript for interacting with the map. All of that logic is handled by the Flask app and the functions.py file. This is just a temporary placeholder.


Testing the Flask application locally before deploying it to GitHub:

**1. Set up a Virtual Environment (Highly Recommended):**

   *   **Why:** Virtual environments isolate the project's dependencies. This prevents conflicts with other Python projects.
   *   **How:**
      ```bash
      # Navigate to the project directory (fire-weather-app)
      cd fire-weather-app

      # Create a virtual environment 
      python3 -m venv venv

      # Activate the virtual environment:
      #   On macOS/Linux:
      source venv/bin/activate
      #   On Windows (cmd.exe):
      venv\Scripts\activate.bat
      #   On Windows (PowerShell):
      venv\Scripts\Activate.ps1
      ```
   *   The command prompt should now show `(venv)` at the beginning, indicating the virtual environment is active.  *Always* worked within the activated virtual environment when developing and testing.

**2. Install Dependencies:**

   *   Make sure to be in the project directory (`fire-weather-app`) and that the virtual environment is activated.
   *   Install the required packages using `pip`:
      ```bash
      pip install -r requirements.txt
      ```

**3. Run the Flask App (Development Server):**

   *   Still in the project directory with the virtual environment active, run:
      ```bash
      python app.py
      ```
   *   This will start Flask's built-in development server.  This is in the terminal:
      ```
       * Serving Flask app 'app'
       * Debug mode: on
       * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
       * Restarting with stat
       * Debugger is active!
       * Debugger PIN: ...
      ```
      * **Debug Mode:** `debug=True` is important during development. It provides helpful error messages in the browser and automatically reloads the server when there are code changes. *Do not* use `debug=True` in production.

**4. Access the App in Your Browser:**

   *   Open a web browser (Chrome, Firefox, etc.).
   *   Go to the address shown in the terminal output.  Usually, it's `http://127.0.0.1:5000` (or `http://localhost:5000`).

**5. Test Thoroughly:**

   *   **Initial Load:** The page should load, showing the date input field and the "Generate Map" button. The default date (today) should be pre-filled.
   *   **Default Map:** Click "Generate Map" *without* changing the date.  The map for the current day should appear.  Check that:
      *   The map is centered on Los Angeles.
      *   The LA County boundary is displayed.
      *   The color-coded fire index data points are visible.
      *   The popups work when you click on the data points.
      *   The colormap legend is displayed at the bottom.
   *   **Date Selection:**
      *   Select different dates using the date picker.  Click "Generate Map" for each date.
      *   Verify that the map updates to reflect the data for the selected date.
      *   Test some dates in the past, and if possible, test a date far enough in the past that the weather data might be cached (to test your caching mechanism).
   *   **Error Handling:**
      *   Enter an invalid date format (e.g., "abc") in the date input.  Click "Generate Map".  There should be an "Invalid date format" error message.
      *   Checked to see what happens if the Open-Meteo API is temporarily unavailable?  (This was simulated by temporarily disconnecting from the internet *after* some data has been cached).  The app was able to handle errors gracefully, ideally displaying a message to the user.  It shouldn't crash.
   *   **Inspect HTML (Developer Tools):**
      *   Openned the browser's developer tools.
      *   Went to the "Elements" or "Inspector" tab.
      *   Examined the generated HTML for the map.  Made sure that the `<iframe>` containing the map is correctly embedded, and the colormap `<div>` is present.  Checked for any errors in the browser's console.
      *    Went to the Network tab. Clicked "Generate Map." Saw network requests. Clicked on a request. Checked the Headers tab. Saw the URL for the request. Checked the Preview / Response tabs. They displayed what was returned from the request. 
   *   **Responsiveness:** Resized the browser window.  The map adjusted to different screen sizes.

**6. Deactivate the Virtual Environment (When Done):**

   *   Deactivated the virtual environment:
      ```bash
      deactivate
      ```

**Later Considerations During Testing:**

*   **Caching:** Tested how caching works.  After fetching data for a specific date, tried fetching it again.  The second request was faster because the data is loaded from the cache.
* **API Limits:** Being mindful of the Open-Meteo API's usage limits, I avoided making excessive requests during testing. The caching mechanism helps with this.

